In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-base")  
model = AutoModelForSeq2SeqLM.from_pretrained("VietAI/vit5-base").to('cuda')


In [6]:
tokens = tokenizer.encode("Bầu trời hôm nay rất đẹp và", return_tensors="pt").cuda()
output = model.generate(
    tokens, max_new_tokens=300, use_cache = True # by default is set to True
)
output_text = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

In [7]:
output_text

'* và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất nhiều mây. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời hôm nay rất đẹp và rất đẹp. Bầu trời rất đẹp và rất đẹp. Bầu trời rất đẹp và rất đẹp. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'

In [9]:
import torch

input_seq_length = 5
dim_model = 10

input_ids_emb = torch.randn(input_seq_length, dim_model)
W_q = torch.randn(dim_model, dim_model)
W_k = torch.randn(dim_model, dim_model)
W_v = torch.randn(dim_model, dim_model)

Q = input_ids_emb @ W_q
K = input_ids_emb @ W_k
V = input_ids_emb @ W_v

In [10]:
import torch.nn.functional as F
import math

d_k = K.shape[-1]
attention_scores = (Q @ K.T) / math.sqrt(d_k)

# Lower triangular mask to prevent future token access
causal_mask = torch.tril(torch.ones(input_seq_length, input_seq_length))
masked_scores = attention_scores.masked_fill(causal_mask == 0, float('-inf'))

attention_weights = F.softmax(masked_scores, dim=-1)
output = attention_weights @ V

In [11]:
new_token_emb = torch.randn(1, dim_model)
extended_input = torch.cat([input_ids_emb, new_token_emb], dim=0)

Q_ext = extended_input @ W_q
K_ext = extended_input @ W_k
V_ext = extended_input @ W_v


In [12]:
torch.testing.assert_close(K, K_ext[:input_seq_length]) # test pass
torch.testing.assert_close(V, V_ext[:input_seq_length]) # test pass

In [ ]:
def forward(self, x, cos, sin, attention_mask=None, block_kv_cache=None):
    is_prefill = block_kv_cache is None
    B, T_curr, C = x.size()

    # Project inputs to Q, K, V
    q_curr, k_curr, v_curr = project_current_tokens(x)
    q, k_rotated = apply_rotary_pos_embd(q_curr, k_curr, cos, sin)

    if not is_prefill and block_kv_cache['key'] is not None:
        # Append new keys and values to the cache
        k = torch.cat([block_kv_cache['key'], k_rotated], dim=2)
        v = torch.cat([block_kv_cache['value'], v_curr], dim=2)
    else:
        # First pass (prefill) — no cache
        k, v = k_rotated, v_curr

    block_kv_cache = {'key': k, 'value': v}
    return attention_output, block_kv_cache
